In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

torch.cuda.set_device(0)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH = "../input/"
sz = 224
arch = resnet101
bs = 58

In [3]:
label_csv = f'{PATH}train.csv'
n = len(list(open(label_csv))) - 1 # header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [4]:
label_df = pd.read_csv(label_csv)

In [5]:
label_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [6]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}train.csv', test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, tfms=tfms, bs=bs)

In [7]:
import pathlib
data.path = pathlib.Path('.')

In [8]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /tmp/.torch/models/resnet101-5d3b4d8f.pth
100%|██████████| 178728960/178728960 [00:07<00:00, 22711535.82it/s]


  0%|          | 0/350 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


100%|██████████| 138/138 [02:20<00:00,  1.80it/s]


In [9]:
learn.fit(0.03, 5)

epoch      trn_loss   val_loss   accuracy                   
    0      5.662462   5.432014   0.385252  
    1      5.148924   5.189933   0.390181                   
    2      4.695049   5.034865   0.396885                   
    3      4.345202   4.951031   0.401025                   
    4      3.951834   4.958438   0.40418                    



[4.958438167812696, 0.4041798075584677]

In [10]:
learn.fit(0.02, 5)

epoch      trn_loss   val_loss   accuracy                   
    0      3.327237   4.884673   0.410095  
    1      2.974542   4.890704   0.412658                   
    2      2.628994   5.063364   0.41108                    
    3      2.344879   5.062496   0.415418                   
    4      2.053452   5.108383   0.411869                   



[5.1083832959459405, 0.41186908142470785]

In [11]:
learn.precompute = False

In [12]:
learn.fit(0.02, 5, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                   
    0      3.421908   5.024417   0.419361  
    1      3.326198   5.004565   0.419164                   
    2      3.100337   4.948603   0.422516                   
    3      3.038393   4.926874   0.42291                    
    4      2.964419   4.935752   0.421333                   



[4.93575198229179, 0.42133280491509273]

In [13]:
learn.unfreeze()

In [ ]:
lrf=learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
lr=1e-4

In [14]:
lrs=np.array([1e-5,1e-4,1e-3])

In [ ]:
learn.fit(lrs, 5, cycle_len=1)

  0%|          | 0/350 [00:00<?, ?it/s]

In [ ]:
learn.fit(lrs, 5, cycle_len=1, cycle_mult=2)

In [ ]:
learn.sched.plot_loss()

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

accuracy_np(probs,y)

In [ ]:
lrs=np.array([lr/9,lr/3,lr])
learn.fit(lrs, 5, cycle_len=1)

In [ ]:
learn.set_data(data1)
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('224_pre')

learn.load('224_pre')